# Example of automated sizing with pvpumpingsystem

This example follows on from the two that relate more specifically to simulation, namely 'simulation_tunis_basic.ipynb' and 'simulation_tunis_advanced.ipynb'. The concepts already covered in these examples are not discussed again here.

The sizing presented here is a sizing that chooses the best combination of motor-pump and PV module to reduce the life cycle cost of a system. It first computes the number of PV modules required for each combination to pump enough water in order to respect the maximum water shortage probability (named llp). Then it selects the combination with the lowest net present value.

Numerous criteria and algorithms can be used for sizing such systems. The module 'sizing.py' leaves room for many improvements, extension and new functions. Do not hesitate to share once you have developed the sizing method that suits you best!

In [1]:
import pvlib

import pvpumpingsystem.pump as pp
import pvpumpingsystem.pipenetwork as pn
import pvpumpingsystem.consumption as cs
import pvpumpingsystem.pvpumpsystem as pvps
import pvpumpingsystem.pvgeneration as pvgen
import pvpumpingsystem.reservoir as rv
import pvpumpingsystem.mppt as mppt
from pvpumpingsystem import sizing

## Pump and PV module databases

Three pumps are available here. The user wants to find the one which fits best for the application. First the 3 pumps must be imported.

Note that the motor-pumps coming from 'sunpump' follow a naming convention: the 3 numbers are respectively the flow rate in GPM, the Head in feet, and the voltage in V at the rated operating point. Note that the rated voltage is also the maximum input voltage for the pump.

In [2]:
pump_1 = pp.Pump(
    path="../../pvpumpingsystem/data/pump_files/SCB_10_150_120_BL.txt")

pump_2 = pp.Pump(
    path="../../pvpumpingsystem/data/pump_files/SCB_10_150_180_BL.txt")

# For the pump remember that the details given in the text file can be
# overwritten in its definition: for example the price is overwritten here.
pump_3 = pp.Pump(
    path="../../pvpumpingsystem/data/pump_files/SCS_12_127_60_BL.txt",
    price=1300)

# The database must be provided under the form of a list for the sizing:
pump_database = [pump_1,
                 pump_2,
                 pump_3]

c:\users\tanguy\documents\code\src\pvpumpingsystem\pvpumpingsystem\pump.py:94: UserWarning: price attribute overwritten.
  warnings.warn('price attribute overwritten.')


The user has the choice between two different PV modules here. The names of these must be given in a list as well:

In [3]:
pv_database = ['Kyocera solar KU270 6MCA',
               'Canadian Solar CS5C 80M']

As it relies on a factorial design, the computation time can increase rapidly if the databases are too long. This sizing is therefore more relevant when applied on relatively small databases, unless you have time...

## Rest of the system

Then, the rest of the system is defined. This part is therefore fixed and will not be sized.

The sizing module provides some functions to shorten the computation time. The one used here keeps only the worst month (the month with the lowest global horizontal irradiance) in the weather data. This way, the computation time is approximately divided by 12.

In [4]:
# Weather input
weather_data, weather_metadata = pvlib.iotools.epw.read_epw(
    '../../pvpumpingsystem/data/weather_files/TUN_Tunis.607150_IWEC.epw',
    coerce_year=2005)
weather_data = sizing.shrink_weather_worst_month(weather_data)


Then the weather data can be imported into a PVGeneration object. In this object, the 'pv_module_name' attribute is not important but cannot be None. Also, there is no need to give a number of modules in parallel or in series as it is automatically sized later in the process.

In [5]:
# PV generator parameters
pvgen1 = pvgen.PVGeneration(
            # Weather data
            weather_data_and_metadata={
                    'weather_data': weather_data,
                    'weather_metadata': weather_metadata},  # to adapt:

            # PV array parameters
            pv_module_name=pv_database[0],
            price_per_watt=2.5,  # in US dollars
            surface_tilt=45,  # 0 = horizontal, 90 = vertical
            surface_azimuth=180,  # 180 = South, 90 = East
            albedo=0.3,  # between 0 and 1
            racking_model='open_rack',  # or'close_mount' or 'insulated_back'

            # Models used
            orientation_strategy=None,  # or 'flat' or 'south_at_latitude_tilt'
            clearsky_model='ineichen',
            transposition_model='haydavies',
            solar_position_method='nrel_numpy',
            airmass_model='kastenyoung1989',
            dc_model='desoto',  # 'desoto' or 'cec'.
            ac_model='pvwatts',
            aoi_model='physical',
            spectral_model='no_loss',
            temperature_model='sapm',
            losses_model='no_loss'
            )

The rest of the system does not differ from the simulation examples seen before: 

In [6]:
# MPPT
mppt1 = mppt.MPPT(efficiency=0.96,
                  price=1000)

# Pipes set-up
pipes1 = pn.PipeNetwork(h_stat=20,  # vertical static head [m]
                       l_tot=100,  # length of pipes [m]
                       diam=0.05,  # diameter of pipes [m]
                       material='plastic')

# Reservoir
reservoir1 = rv.Reservoir(size=5000,  # [L]
                          water_volume=0,  # [L] at beginning
                          price=(1010+210))  # 210 is pipes price


The consumption input is maybe the most important. It represents the need of the final user.

In [7]:
# Consumption input
# represents 7200L/day
consumption_data = cs.Consumption(constant_flow=5)  # in L/min


Definition of the system. PVGeneration object must be given even if some attributes will be changed afterward by the sizing function. Pump attribute can be kept as None.

In [8]:

pvps_fixture = pvps.PVPumpSystem(pvgen1,
                                 None,
                                 motorpump_model='arab',
                                 coupling='mppt',
                                 mppt=mppt1,
                                 reservoir=reservoir1,
                                 pipes=pipes1,
                                 consumption=consumption_data)


## Result

Eventually, run the function to get the best system. The attribute 'llp_accepted' is important here as it will be used to determine the number of PV module required for each combination. It represents the Loss of Load Probability (=Water shortage probability). For systems aimed at providing critical domestic water it should be typically kept under 0.05 (5%), and for irrigation systems it can be more flexible (depending on the crop, the soil, ...).
In order to compute the net present value (`npv`), the financial parameters must be given as keyword arguments here as well.
In the present case, it should take around 30 seconds to find the best combination.

In [9]:
selection, total = sizing.sizing_minimize_npv(pv_database,
                                              pump_database,
                                              weather_data,
                                              weather_metadata,
                                              pvps_fixture,
                                              llp_accepted=0.05,
                                              M_s_guess=5,
                                              discount_rate=0.05,
                                              opex=100)

Research of best combination:   0%|                                                              | 0/2 [00:00<?, ?it/s]

module: Kyocera_Solar_KU270_6MCA / pump: SCB_10_150_120_BL / M_s: 5 / llp: 0.005967761904012374 / npv: 11698.04544037404
module: Kyocera_Solar_KU270_6MCA / pump: SCB_10_150_120_BL / M_s: 4 / llp: 0.02220801779686927 / npv: 10888.015440374038
module: Kyocera_Solar_KU270_6MCA / pump: SCB_10_150_120_BL / M_s: 3 / llp: 0.1015403764354007 / npv: 10077.98544037404
module: Kyocera_Solar_KU270_6MCA / pump: SCB_10_150_120_BL / M_s: 4 / llp: 0.02220801779686927 / npv: 10888.015440374038
module: Kyocera_Solar_KU270_6MCA / pump: SCB_10_150_180_BL / M_s: 5 / llp: 0.03539426523297491 / npv: 11861.586347147155
module: Kyocera_Solar_KU270_6MCA / pump: SCB_10_150_180_BL / M_s: 4 / llp: 0.05186642702867146 / npv: 11051.556347147154
module: Kyocera_Solar_KU270_6MCA / pump: SCB_10_150_180_BL / M_s: 5 / llp: 0.03539426523297491 / npv: 11861.586347147155
module: Kyocera_Solar_KU270_6MCA / pump: SCS_12_127_60_BL / M_s: 5 / llp: 0.007479875918178493 / npv: 12152.823578386948
module: Kyocera_Solar_KU270_6MCA /

Research of best combination:  50%|███████████████████████████                           | 1/2 [00:07<00:07,  7.39s/it]

module: Kyocera_Solar_KU270_6MCA / pump: SCS_12_127_60_BL / M_s: 4 / llp: 0.02109467026317708 / npv: 11342.793578386947
module: Canadian_Solar_Inc__CS5C_80M / pump: SCB_10_150_120_BL / M_s: 5 / llp: 0.47807788062984186 / npv: 8850.145440374039
module: Canadian_Solar_Inc__CS5C_80M / pump: SCB_10_150_120_BL / M_s: 6 / llp: 0.36041991174164234 / npv: 9090.59544037404
module: Canadian_Solar_Inc__CS5C_80M / pump: SCB_10_150_120_BL / M_s: 7 / llp: 0.28599183977330006 / npv: 9331.04544037404
module: Canadian_Solar_Inc__CS5C_80M / pump: SCB_10_150_120_BL / M_s: 8 / llp: 0.22212701835287782 / npv: 9571.49544037404
module: Canadian_Solar_Inc__CS5C_80M / pump: SCB_10_150_120_BL / M_s: 9 / llp: 0.1629358534013387 / npv: 9811.945440374038
module: Canadian_Solar_Inc__CS5C_80M / pump: SCB_10_150_120_BL / M_s: 10 / llp: 0.11193007478607034 / npv: 10052.395440374039
module: Canadian_Solar_Inc__CS5C_80M / pump: SCB_10_150_120_BL / M_s: 11 / llp: 0.07645672295472879 / npv: 10292.84544037404
module: Canad

Research of best combination: 100%|██████████████████████████████████████████████████████| 2/2 [00:21<00:00, 10.61s/it]

module: Canadian_Solar_Inc__CS5C_80M / pump: SCS_12_127_60_BL / M_s: 12 / llp: 0.029852077900683456 / npv: 10988.073578386948


In [10]:
# Final result:
print('Cheapest PV pumping system:\n', selection)

Cheapest PV pumping system:
    M_p   M_s       llp          npv               pump  \
3  1.0  13.0  0.030033  10773.74544  SCB_10_150_120_BL   

                      pv_module  
3  Canadian_Solar_Inc__CS5C_80M  
